In [12]:
import requests as rq
import tensorflow as tf
import csv
#apt-get
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, f_classif
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam

In [3]:
#So basicalee the way it works is you call the method with the csv files path in as a parameter
#make sure when making the csv file to make the columns as the input categories and the row as the country/subregion
#That way the data will output arrays where each index is a statistic


def parse_data(PATH, dictionary, firstCountry, loan_loss_rates):
    with open(PATH, 'r') as file: 
        fileReader = csv.reader(file)
        intData = []
    
        currCountry = firstCountry
        currYear = "YR2010"
        for row in fileReader:
            
            #some rows dont have country data like the first one so will not iterate across and make new keyval
            if(data.get(str(row[1])) is not None):
                #adding all the values from the row to this list
                currData = []
                data.get(str(row[0])).setdefault(str(row[2]), [])
                #done gathering the data for each subregion of the country can compress the 2d array and start new dataset
                if(currCountry != row[2]):
                    
                    dataList = compress_data(intData)
                    #print(dataList)
                    intData = []
                    data.get(currYear).update({currCountry: dataList})
                    currCountry = row[2]
                    currYear = str(row[0])

                for i  in range(6, len(row)):
                    if(i == 98):
                        if row[i] == '..':
                            loan_loss_rates.get(str(row[0])).append(0)
                        else:
                            loan_loss_rates.get(str(row[0])).append(float(row[i]))
                    else:
                        if row[i] == '..':
                            currData.append(None)
                            #btw i append none here cuz temporarily to calculate averages i need to distiguish between None and 0
                        else:
                            currData.append(float(row[i]))
                intData.append(currData)
                        
#Since the data for each country is stored in its subregions and we only care about the data for the country,
#I take the 2d matrix of data, row is each subregion and data is columns, add the values for each data point and
#divide by the number of times it is not null (average each data point) and store as single list            
def compress_data(largeData):
    numpyArray = np.array(largeData)
    transpose = numpyArray.T
    transpose_list = transpose.tolist()
    returnList = []

    for listVal in transpose_list:
        nonNulls = 0;
        avg = 0;
        for val in listVal:
            if(val is not None):
                avg += val
                nonNulls +=1
        if(nonNulls != 0):
            avg /= nonNulls
            returnList.append(avg)
        else:
            returnList.append(0)
    return returnList
        
paths = ['/work/Data' + str(i) + ".csv" for i in range(7)]
firstCountries = ["Afghanistan", "Nicaragua", "Togo", "Bulgaria", "Central African Republic", "Congo, Dem. Rep.", "Ethiopia"]

loan_loss_rates = {}
data = {}
for i in range(10):
    yr = str(2010 + i)
    data.update({"YR" + yr: {}})
    loan_loss_rates.update({"YR" + yr: []})

for i in range(len(paths)):
    parse_data(paths[i], data, firstCountries[i], loan_loss_rates)

#this loop gathers all the data
#for i in range(len(paths)):
    #parse_data(paths[i], data, firstCountries[i])

#thats the data from first csv
print(len(data["YR2010"]))





76


In [4]:
for key_dict in data.keys():
    print(key_dict + "\n")
    for key in data[key_dict].keys():
        print(key)

YR2010

Afghanistan
Albania
Angola
Argentina
Armenia
Azerbaijan
Bangladesh
Belarus
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Brazil
Nicaragua
Niger
Nigeria
Pakistan
Peru
Philippines
Russian Federation
Rwanda
Senegal
Sierra Leone
South Africa
South Sudan
Sri Lanka
Tajikistan
Tanzania
Togo
Uganda
Uzbekistan
Vietnam
West Bank and Gaza
Yemen, Rep.
Zambia
Zimbabwe
Bulgaria
Burkina Faso
Burundi
Cambodia
Central African Republic
Chad
Chile
China
Colombia
Congo, Dem. Rep.
Congo, Rep.
Costa Rica
Cote d'Ivoire
Croatia
Dominican Republic
Ecuador
Egypt, Arab Rep.
El Salvador
Ethiopia
Guatemala
Honduras
India
Indonesia
Iraq
Jamaica
Jordan
Kazakhstan
Kenya
Kyrgyz Republic
Lao PDR
Madagascar
Mali
Mexico
Moldova
Mongolia
Morocco
Mozambique
Myanmar
Nepal
YR2011

Afghanistan
Albania
Angola
Argentina
Armenia
Azerbaijan
Bangladesh
Belarus
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Brazil
Nicaragua
Niger
Nigeria
Pakistan
Peru
Philippines
Russian Federation
Rwanda
Senegal
Sierra Leone
South

In [5]:

keys = []
for key in data["YR2014"]:
    keys.append(key)
keys.sort()
for key in keys:
    print(key)

print(len(keys))

Afghanistan
Albania
Angola
Argentina
Armenia
Azerbaijan
Bangladesh
Belarus
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Brazil
Bulgaria
Burkina Faso
Burundi
Cambodia
Central African Republic
Chad
Chile
China
Colombia
Congo, Dem. Rep.
Congo, Rep.
Costa Rica
Cote d'Ivoire
Croatia
Dominican Republic
Ecuador
Egypt, Arab Rep.
El Salvador
Ethiopia
Guatemala
Honduras
India
Indonesia
Iraq
Jamaica
Jordan
Kazakhstan
Kenya
Kyrgyz Republic
Lao PDR
Madagascar
Mali
Mexico
Moldova
Mongolia
Morocco
Mozambique
Myanmar
Nepal
Nicaragua
Niger
Nigeria
Pakistan
Peru
Philippines
Russian Federation
Rwanda
Senegal
Sierra Leone
South Africa
South Sudan
Sri Lanka
Tajikistan
Tanzania
Togo
Uganda
Uzbekistan
Vietnam
West Bank and Gaza
Yemen, Rep.
Zambia
Zimbabwe
76


In [6]:
#loan retention 

data_df = pd.read_csv("./loan_retention.csv")

columns = ["Country Name"]
for i in range(8):
    yr = str(2011 + i)
    columns.append(yr)
data_df = data_df[columns]
data_df = data_df.dropna()
#print(data_df.head(20))
# print(data_df.shape)
outputs = {}
for i in range(len(data_df)):
    vals = []
    for j in range(1,len(columns)):
        vals.append(data_df.iloc[i][columns[j]])
    country = str(data_df.iloc[i]["Country Name"])
    outputs.update({country: vals})
print(outputs)
#keys are the country
print(outputs.keys())








{'Afghanistan': [4.696817825, 4.958901404, 4.852287048, 7.781090896, 12.05361798, 11.07437896, 12.20183019, 8.886660184], 'Angola': [2.014968215, 4.350118845, 7.97676918, 10.19053207, 10.61123397, 11.28508581, 25.83606103, 23.24050574], 'Albania': [18.76933916, 22.4938973, 23.49277786, 22.80273406, 18.16857405, 18.26995219, 13.23253338, 11.084582], 'United Arab Emirates': [6.086295813, 7.129651752, 6.655645965, 5.636720012, 5.02708123, 5.071833428, 5.295286504, 5.610436526], 'Argentina': [1.395598406, 1.734548721, 1.732582431, 1.990064383, 1.736841818, 1.836605544, 1.830363712, 3.108572249], 'Armenia': [3.421091513, 3.669222171, 4.485338923, 6.969750428, 7.949716684, 6.724111251, 5.432317779, 4.754398251], 'Australia': [1.968386346, 1.701891508, 1.365174882, 1.013844499, 0.888725499, 0.947676353, 0.85701375, 0.901855194], 'Austria': [2.706815505, 2.813364766, 2.866541695, 3.473374315, 3.392837284, 2.695217415, 2.368895442, 1.882428564], 'Belgium': [3.302301757, 3.738254507, 4.244977405

In [9]:
#shrinking the input values and normalizing
def normalize(input_values):
    return preprocessing.scale(input_values)

#data is the old list of data with the extra (nonshrinked) data 
#uncomment the below two pops when running code again
data.pop("YR2019")
data.pop("YR2010")
#new_data is the datw which is shrinked by only taking values from data with outputs
years = data.keys()
other_countries = data["YR2011"].keys()
all_countries = outputs.keys()

#now only including countries with all years of outputs
non_subsets = []
for country in other_countries:
    if country not in all_countries:
        non_subsets.append(country)

for year in years:
    for non in non_subsets:
        data[year].pop(non)
total_countries = data["YR2011"].keys()

non_subsets = []
for country in all_countries:
    if country not in total_countries:
        non_subsets.append(country)

for non in non_subsets:
    outputs.pop(non)
total_countries = data["YR2011"].keys()
total_countries_2 = outputs.keys()

print(total_countries == total_countries_2)

print(data["YR2011"]["Afghanistan"])
print(data["YR2011"]["Albania"])

True
[0, 0, 0, 0, 0, 1169571.0, 10.351428571428572, 0, 0, 32331232.14285714, 520.25, 114.0, 520.25, 114.0, 3693177.714285714, 10410618.25, 461.2307692307692, 101.88692307692305, 22989.75, 24184.545454545456, 461.2307692307692, 101.88692307692305, 13.714285714285714, 55.48666666666667, 167.71428571428572, 61.166666666666664, 11889959.333333334, 25.228750000000005, 13861310.625, 0, 182.71428571428572, 204.16666666666666, 3.874285714285714, 61.285714285714285, 61.285714285714285, 23561091.0, 28.10625, 13.636249999999999, 93065068.0, 0, 0, 8004989.0, 83884.0, 15842210.0, 11534835.0, 4307375.0, 122546.0, 3607022.25, 221712.5, 0, 0, 229532.8, 3.1950000000000003, 926427.2857142857, 4474878.0, 20.475714285714286, 4007069.5125000007, 10944196.785714285, 71.86375000000001, 7815830.0, 0.0, 4122373.0, 11616515.75, 2094262.0, 0.0, 6220563.5, 20740.833333333332, 0, 0, 264849.5, 37321.0, 227528.5, 46755.25, 180773.25, 1190.0, 1675341.111111111, 1030593.25, 6703093.888888889, 12009797.0, 4809298.25, 1

In [ ]:
#putting data into numpy arrays
def normalize(input_values):
    scaler = preprocessing.StandardScaler()
    return scaler.fit_transform(input_values)

def normalize_outputs(outputs):
    return preprocessing.scale(outputs)

sorted_keys = sorted(outputs.keys())
print(sorted_keys)
print(len(sorted_keys))
array_2d_inputs = []
array_outputs = []
#output vector is a row vector

# count = 0
# for key in data.keys():
#     year_data = data.get(key)
#     for sorted_key in sorted_keys:
#         array_2d_inputs.append(year_data.get(sorted_key))
#         array_outputs.append(outputs.get(sorted_key)[count])
#     count +=1

for sorted_key in sorted_keys:
    new_input = []
    for key in data.keys():
        new_input.append(data[key][sorted_key])
    array_2d_inputs.append(new_input)
    array_outputs.append(outputs[sorted_key][-1])


normalized_array = [normalize(arr) for arr in array_2d_inputs ]
numpy_inputs = np.array(array_2d_inputs)
#input array (number of training examples x number of features )

#for now use the full window of all 8 years to make a prediction, later can make window smaller and generate
#more training examples
array_outputs = normalize_outputs(array_outputs)
numpy_outputs = np.array(array_outputs)

print(np.shape(numpy_outputs))
print(np.shape(numpy_inputs))




['Afghanistan', 'Albania', 'Angola', 'Argentina', 'Armenia', 'Bangladesh', 'Belarus', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Cambodia', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Congo, Rep.', 'Costa Rica', 'Croatia', 'Dominican Republic', 'Ecuador', 'El Salvador', 'Guatemala', 'Honduras', 'India', 'Indonesia', 'Kazakhstan', 'Kenya', 'Kyrgyz Republic', 'Madagascar', 'Mexico', 'Moldova', 'Nicaragua', 'Nigeria', 'Pakistan', 'Peru', 'Philippines', 'Russian Federation', 'Rwanda', 'South Africa', 'Sri Lanka', 'Tanzania', 'Uganda', 'Uzbekistan', 'Vietnam', 'West Bank and Gaza', 'Zambia']
49
49


8

270
270
270
270
270
270
270
270
8

270
270
270
270
270
270
270
270
8

270
270
270
270
270
270
270
270
8

270
270
270
270
270
270
270
270
8

270
270
270
270
270
270
270
270
8

270
270
270
270
270
270
270
270
8

270
270
270
270
270
270
270
270
8

270
270
270
270
270
270
270
270
8

270
270
270
270
270
270
270
270
8

270
270
270
270
270
270
270
27

In [ ]:
# recursive feature elimination
#k_best = SelectKBest(score_func=f_classif, k=150)
# # fit on train set
# fit = k_best.fit(numpy_inputs, numpy_outputs)
# # transform train set
# input_features_reduced = fit.transform(numpy_inputs)

# print(np.shape(input_features_reduced))

ValueError: Found array with dim 3. Estimator expected <= 2.

In [ ]:
#input array is  (number of training examples x number of features ) named "input_features_reduced"
#output is (labels x 1) and named "numpy_outputs"

In [15]:
X = numpy_inputs
Y = numpy_outputs
model = Sequential()
model.add(LSTM(np.shape(X)[0], input_shape=(np.shape(X)[1], np.shape(X)[2])))
model.add(Dense(1))
model.add(Activation('relu'))


In [ ]:
optimizer = Adam(lr=0.01)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0, batch_size=49, epochs=20, shuffle=True).history

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=83e81097-0ea7-4ea9-b6cb-50a8706e3fdb' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>